In [1]:
import pandas as pd
import numpy as np
import math 
import random 

In [2]:
# read in data
df = pd.read_excel('Tutor Allocation 2023-24 - Lars 1 Copy.xlsx', sheet_name = ['Courses', 'S2 Course Lookups'])

### Making Time Slots

In [3]:
## Get list of am and pm slots

ams = []
pms = []
for i in range(0, 19*5):
    for j in range(6):
        if i in range(j*19, (j*19)+6):  # list of am slots
            ams.append(i)
    if i not in ams:
        pms.append(i)  # list of pm slots 

# print(len(ams), len(pms)) # am = 30, pm = 65

# Mon AM  ams[0:6]  # 0-5
# Mon PM  pms[0:13]  # 6-18

# Tues AM ams[6:12]  # 19-24
# Tues PM  pms[13:26]  # 25-37

# Weds AM  ams[12:18]  # 38-43
# Weds PM  pms[26:39]  # 44-56

# Thrus AM  ams[18:24]  # 57-62
# Thurs PM  pms[39:52]  # 63-75

# Fri AM  ams[24:]  # 76-81
# Fri PM  pms[52:]  # 82-94

### Course Information

In [4]:
# load all_courses sheet from full excel file 
all_courses = df['Courses']

In [5]:
## Formatting 

# load all_courses sheet from full excel file 
all_courses = df['Courses']

# drop unnecessary/unhelpful columns 
all_courses.drop(columns = ['Acronym', 
                        'Course Organiser', 
                        'Course Secretary',
                        'Credits', 
                        'Course info notes',
                        'Live this year? 2022/23',
                        '2023/24',
                        'No ST Required', 
                        'Super Tutor contact hrs for 1 session',
                        'Super Tutor Multiplier',
                        'NYT',
                        'Frequency'], inplace = True)

# drop NaN rows at end of sheet
all_courses.dropna(inplace = True)

# all_courses.shape  # 106 all_courses, 13 columns after dropping 

# list of course names
course_names = all_courses['Course Name'].to_list()  # length 106 all_courses as expected 

# list of course codes
course_codes = all_courses['Code'].to_list()

# set index to normal list of numbers 0-whatever
all_courses.reset_index(inplace = True)

# find shape of df
all_courses.shape # (106, 13)  

(106, 14)

### Workshop Times

In [6]:
## Getting list of workshop times given to us

# load lookups sheet from full excel file 
lookups = df['S2 Course Lookups']  # workshop times of all courses offered in Sem 2

# get rid of Maths Base column because aren't counting that one 
lookups.drop(columns = ['Maths Base Semester 2'], inplace = True)

# drop the row with course numbers
# lookups.drop([0], inplace = True)

# get rid of extra NaN rows at end (all rows that have NaN for all courses,
    # (were there because the whole sheet was mostly empty rows)
lookups.drop(lookups.index[6:-1], inplace = True)

# get list of course names (all courses with workshops scheduled in Sem 2)
lookup_list = lookups.columns.to_list()

# get list of course codes (all courses with workshops scheduled in Sem 2)
lookup_codes = lookups.loc[0,:].to_list()

# len(lookup_list)  # 70 courses
    
# lookups.shape # (7,70)

In [7]:
## Formatting workshops list into time_slots 

# make empty df for course names and workshop times
fr = pd.DataFrame(columns = ['course', 'code', 'info'])

# add info into df
for i in lookup_list:  # goes through all columns (course names)
    for j in range(1,len(lookups)): # len(lookups) = 7 --> each class will have seven entries in info column 
        fr.loc[len(fr)] = [i, lookups[i].iloc[0], lookups[i].iloc[j]]  # course name, course code, ws times 

# drop all NaN values (lots becuase have 7 entries per course and most courses have 1-3 ws times)
fr.dropna(inplace = True)

# separate info column 
k = fr['info'].str.split(expand = True) # split into day, start time, dash, end time
# k.iloc[:,0]  # just the days 
b = k.iloc[:,1].str.split(':', expand = True) # split start time into hours and mins
c = k.iloc[:,3].str.split(':', expand = True) # split end time into hours and mins

# # put them all together into workshop time 
workshops = pd.concat([fr, k.iloc[:,0], b, c], axis = 1)

# get rid of original column of info because it's the wrong format now 
workshops.drop(['info'], axis = 1, inplace = True)

# rename columns 
workshops.columns = ['course', 'code', 'day','start_hour', 'start_min', 'end_hour', 'end_min']

# workshops.shape # (104, 7)

# make hour and min columns into integers instead of strings 
workshops['start_hour'] = workshops['start_hour'].astype(int)
workshops['start_min'] = workshops['start_min'].astype(int)
workshops['end_hour'] = workshops['end_hour'].astype(int)
workshops['end_min'] = workshops['end_min'].str[:2]  # get rid random shit at end of string, only want numbers
workshops['end_min'] = workshops['end_min'].astype(int)

In [8]:
## Get the time slots for each workshop

# function to generate which slot (0-94) each workshop is in  
def get_time_slot(day: str, hour: int, minute: int) -> int:
    days = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4}
    
    start_hour = 9
    start_minute = 0  # 9 AM start slot
    end_hour = 18  # 6 PM end slot
       
    time_offset = (hour - start_hour) * 2 + (minute // 30)
    day_offset = days[day] * 19  # each day has 19 slots
    
    return day_offset + time_offset

# find out start and end slots of all workshops 
start_slot = []
end_slot = []
for i in range(len(workshops)):
    start_val = get_time_slot(day = workshops['day'].iloc[i], hour = workshops['start_hour'].iloc[i],minute= workshops['start_min'].iloc[i])
    start_slot.append(start_val)
    end_val = get_time_slot(day = workshops['day'].iloc[i], hour = workshops['end_hour'].iloc[i],minute= workshops['end_min'].iloc[i])
    if (workshops['end_min'].iloc[i] == 0) or (workshops['end_min'].iloc[i] == 30):
        end_slot.append(end_val -1)
    else: 
        end_slot.append(end_val)
        
# print(len(start_slot), len(end_slot)) # both length 104

# add slot info to workshops df 
workshops['start_slot'] = start_slot
workshops['end_slot'] = end_slot

# calculate how many time slots each workshop takes 
workshops['num_slots'] = workshops['end_slot'] - workshops['start_slot'] + 1

# get rid of redundant stuff (start hour and min, end hour and min)
workshops.drop(columns=['start_hour', 'start_min', 'end_hour', 'end_min'], inplace = True)
    # gives us just course name, day of week, start slot, and end slot

workshop_codes = workshops['code'].unique().tolist() # ============================== NOT SORTED BY ALPHABETICAL COURSES

# workshops.shape  # (104, 6)

In [9]:
## just more information about workshops I guess

# check that all workshops have info in all_courses df. if not, remove them because we can't assign them 
gotta_delete = []
for i in workshop_codes:
    if i not in course_codes:
        gotta_delete.append(i)  # returns list of course codes of classes that we don't have course data for
        # len(gotta_delete)  # 10

# remove them from the list of workshop codes
for i in gotta_delete:
    if i in workshop_codes:
        workshop_codes.remove(i)
        
        
# remove rows of courses we don't have info for from the workshops df
workshops = workshops.loc[~workshops['code'].isin(gotta_delete)]

# sort alphabetically by course name to make life easier later
workshops.sort_values('course', inplace = True)

# reset index so have list 0-whatever
workshops.reset_index(inplace = True)

# workshops.shape  # (93, 6)

# get index in the all_courses df of each course we DO have info for
temp_idx_list = []
for i in workshop_codes: 
    g = all_courses[all_courses['Code'] == i].index
    temp_idx_list.append(g)

idx_list = []
for i in range(len(temp_idx_list)):
    val = temp_idx_list[i][0]
    idx_list.append(val)

# idx_list
        
# list of all courses in sem 2 that have a workshop
workshop_courses = workshops['course'].unique().tolist()  # len = 67
workshop_courses.sort()  # sort alphabetically 

### now have each individual workshop with start slot #, end slot #, and num_slots (duration)

In [10]:
## df of what time slots each worshop is happening for each course
    # so if have multiple ws, will have multiple sets of 1's 

# make df
ws_slots = pd.DataFrame(columns = ['course', 'code'])

# make 95 empty columns 
num_new_cols = 94
for i in range(num_new_cols):
    ws_slots[0] = 0
    ws_slots[i + 1] = 0
# column labels 0-94 are integers

# fill-in courses and codes columns 
ws_slots['course'] = workshop_courses
ws_slots['code'] = workshops['code']

for i in range(len(workshops)):
    title = workshops['course'].iloc[i] # get course name 
    start_val = workshops['start_slot'].iloc[i] # get start slot
    end_val = workshops['end_slot'].iloc[i] # get end slot
    
    for j in range(len(ws_slots)):
        if ws_slots['course'].iloc[j] == title:
            # fill in df with 1's between ws start and end slots
            ws_slots.loc[j, start_val:end_val] = 1

            ws_slots.fillna(0, inplace=True)   

# make columns integer values
ws_slots.loc[:, 0:94] = ws_slots.loc[:, 0:94].astype(int)

# ws_slots.head()

### Relevant Course Info for all Sem 2 Course with a Workshop

In [11]:
# make df of course info for each course that has a workshop
tut_course_info = pd.DataFrame()

# extract info 
names = []
codes = []
group_sizes = []
num_tut_groups = []
week_hours = []
week_nums = []
tut_reqs = []
tmm = []
hours_reqs = []

tut_mark_hours = []
for i in idx_list: 
    # course names
    val1 = all_courses['Course Name'].iloc[i]
    names.append(val1)
    
    # course codes
    vala = all_courses['Code'].iloc[i]
    codes.append(vala)
    
    # size of each tutorial group 
    valb = all_courses['Group Size (max)'].iloc[i]
    group_sizes.append(valb)
    
    # number of tutorial groups per course
    val2 = math.ceil(all_courses['Number of Tutorial Groups'].iloc[i])
    num_tut_groups.append(val2)
    
    # number of tutor hours per week 
    val3 = math.ceil(all_courses['Tutor Hrs p/w'].iloc[i])
    week_hours.append(val3)
    
    # number of weeks of workshops
    val4 = math.ceil(all_courses['No of weeks'].iloc[i])
    week_nums.append(val4)
    
    # number of tutor required (for the whole course)
    val5 = math.ceil(all_courses['No T Required'].iloc[i])
    tut_reqs.append(val5)
    
    # TMM
    val6 = all_courses['Tutor marking multiplier'].iloc[i]
    tmm.append(val6)
    
    # number of hours required per course (sum of all tutor hours for a semester) 
    val7 = math.ceil(all_courses['Hours Required'].iloc[i])
    hours_reqs.append(val7)
    
    # tutor marking hours per course
    val8 = math.ceil(all_courses['Tutor Marking Hours'].iloc[i])
    tut_mark_hours.append(val8)

# put data into df
tut_course_info['course_name'] = names
tut_course_info['code'] = codes 
tut_course_info['group_size'] = group_sizes
tut_course_info['num_tut_groups'] = num_tut_groups
tut_course_info['tut_hrs_week'] = week_hours
tut_course_info['num_weeks'] = week_nums
tut_course_info['num_tut_req'] = tut_reqs
tut_course_info['TMM'] = tmm
tut_course_info['hrs_req'] = hours_reqs
tut_course_info['tut_mark_hours'] = tut_mark_hours

# df.sort_values('column1')
tut_course_info.sort_values('course_name', inplace = True)

# reset index
tut_course_info.reset_index(inplace = True)

# get number of workshops per course counts
ws_counts = []
for i in workshop_courses:
    val = workshops['course'].value_counts()[i]
    ws_counts.append(val)

# add workshop counts column to df
tut_course_info['num_ws'] = ws_counts

# calculate how many tutors are needed in each workshop session 
tut_course_info['num_tut_per_ws'] = np.ceil(tut_course_info['num_tut_req']/tut_course_info['num_ws'])
tut_course_info['num_tut_per_ws'] = tut_course_info['num_tut_per_ws'].astype(int)

# tut_course_info.head()

### Making r and W Variables for Mosel

In [12]:
## make r variable df
# (number of tutors needed per ws slot)

rvar2 = ws_slots.copy()

for i in range(len(rvar2)):
    repl_val = tut_course_info['num_tut_per_ws'].iloc[i]
    
    rvar2.loc[i, :] = rvar2.loc[i,:].replace(1, repl_val)
    
rvar2.head(6)

,course,code,0,1,2,3,4,5,6,7,...,85,86,87,88,89,90,91,92,93,94
0,Algebraic Geometry,MATH11120,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Algebraic Topology,MATH10077,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Applied Dynamical Systems,MATH11140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Applied Statistics,MATH10096,0,0,0,0,0,0,0,0,...,0,9,9,0,0,0,0,0,0,0
4,Bayesian Data Analysis,MATH11175,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,0,0,0
5,Calculus & its Applications,MATH11175,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
## make W variable df 
# (if timeslots t belongs to same WS as timeslots t-1)

W_var = ws_slots.copy()

for i in range(len(workshops)):
    title = workshops['course'].iloc[i]
    start_val = workshops['start_slot'].iloc[i]
    
    for j in range(len(ws_slots)):
        if ws_slots['course'].iloc[j] == title:
            W_var.loc[j, start_val] = 0

### Tutor Availability Info

In [14]:
# read in data
su = pd.read_excel('Tutor Survey Semester 2 Copy.xlsx')

# get smaller df of tutor survey responses, don't focus on preferences, only care abt availability
small_su = su[['Name2', 'Are you available to tutor in Semester 2?', 'Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']]

# rename that stupidly long column name 
small_su = small_su.rename(columns= {'Are you available to tutor in Semester 2?': 'avail'})

# small_su.shape # (115,7)

# drop any tutors who can't work in semester 2
small_su = small_su[small_su['avail'] != 'No']

# get availability into binary variables 
def process_availability(value):
    return pd.Series({"AM": 1 if value in ["AM", "Both"] else 0, 
                      "PM": 1 if value in ["PM", "Both"] else 0})

# make df of binary availiability 
new_data = []
for _, row in small_su.iterrows():
    entry = {"Name2": row["Name2"]}
    for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]:
        availability = process_availability(row[day])
        entry[f"{day}_AM"] = availability["AM"]
        entry[f"{day}_PM"] = availability["PM"]
    new_data.append(entry)

tut_avail = pd.DataFrame(new_data)

# tut_avail.head()

In [15]:
## Make df of tutor availability for each individual time slot 

slot_info = pd.DataFrame(columns = ['tutor'])

num_new_cols = 94
for i in range(num_new_cols):
    slot_info[0] = 0
    slot_info[i + 1] = 0
# column labels 0-94 are integers, not strings
    
# get tutors in df 
slot_info['tutor'] = small_su['Name2']

# slot_info.shape  # (109,96) --> 109 tutors working 

In [16]:
## Fill in tutor availability df with 1 = available, 0 = unavailable 

for i in range(len(tut_avail)):
    
    # Mon AM
    for j in ams[0:6]:
        if tut_avail['Monday_AM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Monday_AM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    # Mon PM
    for j in pms[0:13]:
        if tut_avail['Monday_PM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Monday_PM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
            
    # Tues AM
    for j in ams[6:12]:
        if tut_avail['Tuesday_AM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Tuesday_AM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Tues PM
    for j in pms[13:26]:
        if tut_avail['Tuesday_PM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Tuesday_PM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Weds AM
    for j in ams[12:18]:
        if tut_avail['Wednesday_AM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Wednesday_AM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Weds PM
    for j in pms[26:39]:
        if tut_avail['Wednesday_PM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Wednesday_PM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Thurs AM
    for j in ams[18:24]:
        if tut_avail['Thursday_AM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Thursday_AM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Thurs PM
    for j in pms[39:52]:
        if tut_avail['Thursday_PM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Thursday_PM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Fri AM
    for j in ams[24:]:
        if tut_avail['Friday_AM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Friday_AM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0
    
    # Fri PM
    for j in pms[52:]:
        if tut_avail['Friday_PM'].iloc[i] == 1:
            slot_info[j].iloc[i] = 1
        elif tut_avail['Friday_PM'].iloc[i] == 0:
            slot_info[j].iloc[i] = 0

# cast all columns as integers 
for i in range(95):
    slot_info[i] = slot_info[i].astype(int)

C:\Users\Elie\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Synthesizing Tutor Preference Data

In [17]:
# make list of possible preference ratings
poss_pref_ratings = [1, 0.5, 0]  # 1 = don't want,  0.5 = indifferent,  0 = prefer 

dont_want_probs = [0.7, 0.2, 0.1]

indiff_probs = [0.1, 0.7, 0.2]

high_pref_probs = [0.1, 0.2, 0.7]

In [18]:
# Don't Wants

# set random seed 
random.seed(2025)

# make df template for preferences
prefs_dont_want = pd.DataFrame()
prefs_dont_want['tutor'] = small_su['Name2']

# make columns of course names, fill with NaN values 
for i in ws_slots['course']:
    prefs_dont_want[i] = np.random.choice(a = poss_pref_ratings, size = len(prefs_dont_want), replace = True, p = dont_want_probs)

# prefs_dont_want.head(2)  

In [19]:
# Indifference

# set random see 
random.seed(2025)

# make df template for preferences
prefs_indiff = pd.DataFrame()
prefs_indiff['tutor'] = small_su['Name2']

# make columns of course names, fill with NaN values 
for i in ws_slots['course']:
    prefs_indiff[i] = np.random.choice(a = poss_pref_ratings, size = len(prefs_indiff), replace = True, p = indiff_probs)

# prefs_indiff.head(2)  

In [20]:
# High preferences

# set random see 
random.seed(2025)

# make df template for preferences
prefs_high = pd.DataFrame()
prefs_high['tutor'] = small_su['Name2']

# make columns of course names, fill with NaN values 
for i in ws_slots['course']:
    prefs_high[i] = np.random.choice(a = poss_pref_ratings, size = len(prefs_indiff), replace = True, p = high_pref_probs)

# prefs_high.head(2)  

### Outputting Data for Mosel Code

In [21]:
## tutor names 
print(tut_avail['Name2'].to_list())  # len 

['Tutor 404', 'Tutor 84', 'Tutor 336', 'Tutor 398', 'Tutor 417', 'Tutor 234', 'Tutor 156', 'Tutor 231', 'Tutor 129', 'Tutor 129', 'Tutor 259', 'Tutor 383', 'Tutor 379', 'Tutor 288', 'Tutor 131', 'Tutor 313', 'Tutor 136', 'Tutor 246', 'Tutor 402', 'Tutor 410', 'Tutor 370', 'Tutor 252', 'Tutor 67', 'Tutor 187', 'Tutor 169', 'Tutor 266', 'Tutor 257', 'Tutor 3', 'Tutor 290', 'Tutor 158', 'Tutor 382', 'Tutor 384', 'Tutor 396', 'Tutor 403', 'Tutor 411', 'Tutor 279', 'Tutor 371', 'Tutor 397', 'Tutor 59', 'Tutor 63', 'Tutor 179', 'Tutor 352', 'Tutor 372', 'Tutor 240', 'Tutor 190', 'Tutor 217', 'Tutor 400', 'Tutor 321', 'Tutor 273', 'Tutor 145', 'Tutor 207', 'Tutor 373', 'Tutor 23', 'Tutor 305', 'Tutor 237', 'Tutor 311', 'Tutor 193', 'Tutor 338', 'Tutor 93', 'Tutor 65', 'Tutor 106', 'Tutor 265', 'Tutor 126', 'Tutor 408', 'Tutor 415', 'Tutor 216', 'Tutor 102', 'Tutor 328', 'Tutor 150', 'Tutor 75', 'Tutor 392', 'Tutor 153', 'Tutor 219', 'Tutor 5', 'Tutor 226', 'Tutor 337', 'Tutor 206', 'Tutor 33'

In [22]:
## course names
print(workshop_courses)  # len 57

['Algebraic Geometry', 'Algebraic Topology', 'Applied Dynamical Systems', 'Applied Statistics', 'Bayesian Data Analysis', 'Calculus & its Applications ', 'Classical Mechanics for Mathematicians', 'Computing and Numerics', 'Credit Scoring', 'Data Assimilation', 'Engineering Mathematics/Mathematics for the Natural Sciences 1b', 'Financial Mathematics', 'Financial Risk Theory', 'Fourier Analysis', 'Functional Analysis', 'Fundamentals of Pure Mathematics', 'Galois Theory', 'Geometry of General Relativity', 'Honours Algbra- Skills', 'Honours Algebra', 'Honours Complex Variables', 'Honours Complex Variables - Skills', 'Incomplete Data Analysis', 'Integer and Combinatorial Optimization', 'Interactions in Algebra, Geometry, and Topology', 'Introduction to Number Theory', 'Large Scale Optimization for Data Science', 'Large Scale Optimization for Data Science - Comp Lab', 'Linear Programming, Modelling and Solution', 'Linear Programming, Modelling and Solution Computer Lab', 'Machine Learning in

In [23]:
## time slots 
print(list(range(95)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]


In [24]:
## tutor hours/week/ course (courses)
# len 57 
print(tut_course_info['tut_hrs_week'].to_list())

[1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1]


In [25]:
## W variable (t to t-1 timeslot ws continuity)

all_rows = []

# # Append each row as a list to 'all_rows'
for index, row in W_var.iterrows():
    all_rows.append(row.loc[0:94].tolist())

print(all_rows)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
## Workshop slots (which slots a workshop exists in)

all_rows = []

# # Append each row as a list to 'all_rows'
for index, row in ws_slots.iterrows():
    all_rows.append(row.loc[0:94].tolist())

print(all_rows)


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
## r variable

all_rows = []

# Append each row as a list to 'all_rows'
for index, row in rvar2.iterrows():
    all_rows.append(row.loc[0:94].tolist())

print(all_rows)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
## tutor availability (tut, slots)

tut_timeslots = []

for index, row in slot_info.iterrows():
    tut_timeslots.append(row.loc[0:94].tolist())

print(tut_timeslots)

[[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [29]:
## tutor preferences low

# Create an empty list
few_prefs = []

# Append each row from    ws_slots   as a list to 'ws_timeslot_list'
for index, row in prefs_dont_want.iterrows():
    few_prefs.append(row.iloc[1:-1].tolist())

print(few_prefs)

[[1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 0.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 0.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0], [1.0, 0.0, 0.5, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.5, 1.0, 1.0, 0.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 0.0, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5], [0.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.0, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.

In [30]:
## tutor preferences indifferent

# Create an empty list
indiff_prefs = []

# Append each row from    ws_slots   as a list to 'ws_timeslot_list'
for index, row in prefs_indiff.iterrows():
    indiff_prefs.append(row.iloc[1:-1].tolist())

print(indiff_prefs)

[[0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.0, 1.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.0, 1.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 1.0, 0.5], [0.5, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 1.0, 0.5, 0.5, 0.0, 0.5, 0.5, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 1.0, 0.5, 0.5, 0.0], [0.5, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.0, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.0, 0.

In [31]:
## tutor preferences high prefs 


# Create an empty list
high_prefs = []

# Append each row from    ws_slots   as a list to 'ws_timeslot_list'
for index, row in prefs_high.iterrows():
    high_prefs.append(row.iloc[1:-1].tolist())

print(high_prefs)

[[0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.5, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 1.0, 0.0, 0.0, 0.5, 0.0, 0.0], [0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5], [0.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 1.0, 1.0, 0.